In [1]:
import csv
import random
from sklearn.model_selection import train_test_split

# 한국인 이름 리스트
korean_names = [
    "김준호", "이재민", "박준혁", "최민준", "정도현",
    "강지훈", "조성민", "윤지후", "임민서", "한진우",
    "서시우", "고건우", "문예준", "송현우", "장지호",
    "오지환", "조민호", "유지원", "임성현", "나준우",
    "신우진", "유은우", "강은호", "황예성", "안민규",
    "김재원", "방재현", "오승우", "한현준", "권준영",
    "홍주원", "유영호", "윤준서", "신준", "윤태영",
    "김재영", "이지후", "임주현", "신태현", "서현호",
    "임윤성", "박재우", "권상현", "손현승", "이민재",
    "신찬우", "고진호", "한민혁", "문성준", "손승민",
    "이정우", "박수현", "황재훈", "강민석", "임성우",
    "김예찬", "정성호", "방시현", "오건", "윤건희",
    "류도원", "송재호", "박태균", "임우영", "나주환",
    "전진", "나철민", "임주영", "조예환", "안승호",
    "이수호", "박유준", "최유찬", "박재원", "김동현",
    "이수완", "최상준", "정민수", "신성민", "서민성",
    "장우현", "김시윤", "한재윤", "고준수", "임우성",
    "김태우", "정지우", "박준희", "조지후", "오성윤",
    "정태민", "임재혁", "최영준", "한시훈"
]

# 한국의 지역 목록
korean_locations = ["서울", "부산", "인천", "대구", "대전", "광주", "울산", "세종", "경기", "강원", "충북", "충남", "전북", "전남", "경북", "경남", "제주"]

def generate_random_sms():
    age = random.randint(20, 100)
    name = random.choice(korean_names)
    location = random.choice(korean_locations)
    format_choice = random.randint(1, 5)
    if format_choice == 1:
        sms_text = f"{name}이고 나이는 {age}입니다. {location}에 살고 있습니다."
    elif format_choice == 2:
        sms_text = f"{name},{age},{location}"
    elif format_choice == 3:
        sms_text = f"{location},{age},{name}"
    elif format_choice == 4:
        sms_text = f"{location}/{name}/{age}"
    else:
        sms_text = f"이름은 {name} 입니다. {age}살이고, {location} 거주중입니다."
    return sms_text, {"이름": name, "나이": age, "거주지": location}

filename = "sms_dataset.csv"
data = []

# 데이터셋 생성
for _ in range(100):
    sms_text, answer_data = generate_random_sms()
    data.append({'SMS': sms_text, '이름': answer_data['이름'], '나이': answer_data['나이'], '거주지': answer_data['거주지'], 'label': f"{answer_data['이름']},{answer_data['나이']},{answer_data['거주지']}"})

# 데이터셋을 CSV 파일로 저장
with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['SMS', '이름', '나이', '거주지', 'label']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for item in data:
        writer.writerow(item)

print("Dataset saved to", filename)



Dataset saved to sms_dataset.csv


In [4]:
import csv
import random
import re

# 나이, 이름, 거주지 정보와 접미사를 분리하고, 적절한 BIO 태그를 부여하는 함수
def extract_info_with_suffix(word, age, name, location, suffixes):
    # 나이 정보와 접미사 분리
    if word.startswith(age) and any(word.endswith(suffix) for suffix in suffixes):
        return ('B-age', age), ('O', word.replace(age, '', 1))

    # 이름 정보와 접미사 분리
    if word.startswith(name) and any(word.endswith(suffix) for suffix in suffixes):
        return ('B-name', name), ('O', word.replace(name, '', 1))

    # 거주지 정보와 접미사 분리
    if word.startswith(location) and any(word.endswith(suffix) for suffix in suffixes):
        return ('B-location', location), ('O', word.replace(location, '', 1))

    # 정보와 일치하지 않는 경우 'O' 태그 부여
    return ('O', word), None

# 변환된 데이터를 CSV 파일로 저장하는 함수
def save_transformed_data(filename, data, suffixes):
    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['Word', 'BIO-Tag']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        for sms_text, labels in data:
            words = sms_text.replace(",", " ").replace("/", " ").split()
            name, age, location = labels["이름"], str(labels["나이"]), labels["거주지"]

            for word in words:
                (tag1, core_word), suffix_info = extract_info_with_suffix(word, age, name, location, suffixes)
                writer.writerow({'Word': core_word, 'BIO-Tag': tag1})

                if suffix_info:  # 접미사 정보가 있는 경우 별도로 기록
                    tag2, suffix_word = suffix_info
                    writer.writerow({'Word': suffix_word, 'BIO-Tag': tag2})

# 접미사 리스트
suffixes = [
    "이고", "입니다", "인데", "이에요", "예요", "라고", "이라고", "이었고", "였고", "이라서",
    "이니", "이에게", "께서", "한테", "랑", "이랑", "으로", "로서", "이며", "등",
    "씨", "님", "군", "양", "이자", "처럼", "같이", "만큼", "부터", "까지", "하고", "임", "에게서", "에게", "와", "과",
    "이나", "나", "로써", "이면", "면", "이라", "라", "이었다", "였다", "에서", "이고", "고", "이다", "다", "으로써", "으로서",
    "이든지", "든지", "살", "에"
]

# 데이터셋 생성
data = [(generate_random_sms()) for _ in range(100)]

# 파일 이름 지정 및 데이터 저장
filename = "transformed_sms_dataset_with_full_suffix_handling.csv"
save_transformed_data(filename, data, suffixes)

print(f"Transformed dataset with full suffix handling saved to {filename}")


Transformed dataset with full suffix handling saved to transformed_sms_dataset_with_full_suffix_handling.csv


In [8]:
import csv
import random
import re

def extract_info_with_suffix(word, info, suffixes):
    # 나이, 이름, 거주지 정보와 접미사를 분리하고, 적절한 BIO 태그를 부여
    for suffix in suffixes:
        if word.endswith(suffix):
            # 접미사 제거
            core_word = word.removesuffix(suffix)
            if core_word == info["이름"]:
                return ('B-name', core_word), ('O', suffix)
            elif core_word == str(info["나이"]):
                return ('B-age', core_word), ('O', suffix)
            elif core_word == info["거주지"]:
                return ('B-location', core_word), ('O', suffix)
    # 접미사 없는 경우
    if word == info["이름"]:
        return ('B-name', word), None
    elif word == str(info["나이"]):
        return ('B-age', word), None
    elif word == info["거주지"]:
        return ('B-location', word), None
    return ('O', word), None

def save_transformed_data(filename, data, suffixes):
    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['Word', 'BIO-Tag']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        for sms_text, labels in data:
            words = sms_text.replace(",", " ").replace("/", " ").split()

            for word in words:
                (tag1, core_word), suffix_info = extract_info_with_suffix(word, labels, suffixes)
                writer.writerow({'Word': core_word, 'BIO-Tag': tag1})

                if suffix_info:  # 접미사 정보가 있는 경우 별도로 기록
                    tag2, suffix_word = suffix_info
                    writer.writerow({'Word': suffix_word, 'BIO-Tag': tag2})

# 접미사 리스트
suffixes = [
    "이고", "입니다", "인데", "이에요", "예요", "라고", "이라고", "이었고", "였고", "이라서",
    "이니", "이에게", "께서", "한테", "랑", "이랑", "으로", "로서", "이며", "등",
    "씨", "님", "군", "양", "이자", "처럼", "같이", "만큼", "부터", "까지", "하고", "임", "에게서", "에게", "와", "과",
    "이나", "나", "로써", "이면", "면", "이라", "라", "이었다", "였다", "에서", "이고", "고", "이다", "다", "으로써", "으로서",
    "이든지", "든지", "살", "에", "입니다.", "살이고.", "이며,", "살이며"
]
# 데이터셋 생성
data = [(generate_random_sms()) for _ in range(100)]

# 파일 이름 지정 및 데이터 저장
filename = "transformed_sms_dataset_with_full_suffix_handling.csv"
save_transformed_data(filename, data, suffixes)

print(f"Transformed dataset with full suffix handling saved to {filename}")


Transformed dataset with full suffix handling saved to transformed_sms_dataset_with_full_suffix_handling.csv
